**InsumoProduto.jl: um Breve Tutorial**

Inicialmente, instalemos os pacotes e dados necessários

In [1]:
using LinearAlgebra, PrettyTables,Plots,XLSX,DataFrames,Pkg,HTTP

Agora, instalemos o pacote InsumoProduto.jl

In [2]:
Pkg.add(url="https://github.com/alanleal-econ/InsumoProduto.jl")
using InsumoProduto

    Updating git-repo `https://github.com/alanleal-econ/InsumoProduto.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


0. Leitura dos Dados Necessários

Inicialmente, façamos o download do banco de dados contendo a matriz insumo-produto brasileira de 68 setores para 2018. Este banco de dados foi obtido junto ao NEREUS-FEA/USP.

In [3]:
# URL da planilha no Github
url = "https://raw.githubusercontent.com/alanleal-econ/InsumoProduto.jl/main/dados/MIP%202018%20NEREUS%20Adaptada.xlsx"
# Baixando os dados da planilha
response = HTTP.get(url)
# Salvando o arquivo temporariamente nesta seção
tmp_filename = "temp_file.xlsx"
open(tmp_filename, "w") do file
    write(file, response.body)
end
# Lendo o arquivo temporário
data = XLSX.readxlsx(tmp_filename)
data

XLSXFile("temp_file.xlsx") containing 11 Worksheets
            sheetname size          range        
-------------------------------------------------
           Referência 7x1           A1:A7        
       Matriz Uso SxS 124x84        A1:CF124     
                    Z 68x68         A1:BP68      
                    y 68x1          A1:A68       
                    x 68x1          A1:A68       
                    v 68x1          A1:A68       
                    r 68x1          A1:A68       
                    e 68x1          A1:A68       
                    c 68x1          A1:A68       
                   sp 68x1          A1:A68       
           setores.jl 68x1          A1:A68       


Criando os objetos de interesse agora:

In [4]:
Z=float.(Matrix(data["Z"]["A1:BP68"]))
y=float.(vec(data["y"]["A1:A68"]))
x=float.(vec(data["x"]["A1:A68"]))
v=float.(vec(data["v"]["A1:A68"]))
r=float.(vec(data["r"]["A1:A68"]))
e=float.(vec(data["e"]["A1:A68"]))
c=float.(vec(data["c"]["A1:A68"]))
sp=float.(vec(data["sp"]["A1:A68"]))
Setores=vec(data["setores.jl"]["A1:A68"])

68-element Vector{Any}:
 "Agricultura, inclusive o apoio à agricultura e a pós-colheita"
 "Pecuária, inclusive o apoio à pecuária"
 "Produção florestal; pesca e aquicultura"
 "Extração de carvão mineral e de minerais não-metálicos"
 "Extração de petróleo e gás, inclusive as atividades de apoio"
 "Extração de minério de ferro, inclusive beneficiamentos e a aglomeração"
 "Extração de minerais metálicos não-ferrosos, inclusive beneficiamentos"
 "Abate e produtos de carne, inclusive os produtos do laticínio e da pesca"
 "Fabricação e refino de açúcar"
 "Outros produtos alimentares"
 ⋮
 "Atividades de vigilância, segurança e investigação"
 "Administração pública, defesa e seguridade social"
 "Educação pública"
 "Educação privada"
 "Saúde pública"
 "Saúde privada"
 "Atividades artísticas, criativas e de espetáculos"
 "Organizações associativas e outros serviços pessoais"
 "Serviços domésticos"

1. Inversa de Leontief no Modelo Fechado e Aberto

Aqui, usamos três funções: uma que calcula os coeficientes técnicos e outras duas que calculam a inversa de Leontief do modelo aberto e do modelo fechado. Primeiramente, calculemos a matriz de coeficientes técnicos:

1.1. Matriz A (de coeficientes técnicos) do modelo aberto

In [5]:
A=matrizA(Z,x)
A

68×68 Matrix{Float64}:
 0.0205257    0.034812     0.00414576   …  7.432e-5     0.00145417   0.0
 0.00252489   0.0401885    0.00483232      7.432e-5     0.000221034  0.0
 0.00355495   0.0094565    0.0544758       0.0          1.74501e-5   0.0
 0.000389411  0.00275732   0.00034328      2.47733e-5   2.32668e-5   0.0
 5.77836e-5   0.000105292  2.64061e-5      0.000247733  0.000127967  0.0
 0.0          0.0          0.0          …  0.0          0.0          0.0
 1.25616e-5   1.97422e-5   0.0             0.0          0.0          0.0
 0.0001834    0.0137537    0.00100343      0.000198187  0.00112844   0.0
 0.000135666  0.000572523  5.28123e-5      0.00022296   0.000436252  0.0
 0.00477594   0.0874512    0.0236599       0.00128821   0.00429272   0.0
 ⋮                                      ⋱  ⋮                         
 5.02466e-6   0.0          5.28123e-5      0.00411237   2.32668e-5   0.0
 0.000713502  0.000763364  0.000580935  …  0.00515285   0.00413567   0.0
 5.02466e-6   0.0          0.0 

1.2. Calculando agora a matriz B do modelo aberto. 

In [6]:
B=inversa_leontief(I(size(A)[1])-A)
B

68×68 Matrix{Float64}:
 1.02716      0.0679303    0.0136487    …  0.00230716   0.00842988   -0.0
 0.00322853   1.04879      0.00631778      0.00049703   0.00294043   -0.0
 0.00441402   0.0115811    1.05797         0.000334917  0.000725033  -0.0
 0.00387101   0.00524368   0.000967033     0.000510913  0.000652047  -0.0
 0.022477     0.0214483    0.0114758       0.00561798   0.00873957   -0.0
 0.000412575  0.000809079  0.00027606   …  0.000428151  0.00033401   -0.0
 0.000391052  0.000363408  0.000119575     0.000313433  0.000230335  -0.0
 0.000711904  0.0168321    0.00161408      0.000733416  0.0056871    -0.0
 0.00225137   0.00340821   0.00121674      0.000718514  0.00149642   -0.0
 0.00830622   0.10455      0.0289694       0.00243872   0.0094989    -0.0
 ⋮                                      ⋱  ⋮                         
 0.00192557   0.00210959   0.00113693      0.00592941   0.00248871   -0.0
 0.00288722   0.00292528   0.00139993   …  0.00662495   0.00593763   -0.0
 0.000100929  9.558

1.3. Matriz A do modelo fechado

In [7]:
Af=map(Float64,vcat(hcat(A,c./sum(r)),hcat(r'./x',0)))
Af

69×69 Matrix{Float64}:
 0.0205257    0.034812     0.00414576   …  0.00145417   0.0  0.0232975
 0.00252489   0.0401885    0.00483232      0.000221034  0.0  0.00885341
 0.00355495   0.0094565    0.0544758       1.74501e-5   0.0  0.00576352
 0.000389411  0.00275732   0.00034328      2.32668e-5   0.0  6.54499e-5
 5.77836e-5   0.000105292  2.64061e-5      0.000127967  0.0  0.000197004
 0.0          0.0          0.0          …  0.0          0.0  2.29075e-6
 1.25616e-5   1.97422e-5   0.0             0.0          0.0  2.29075e-6
 0.0001834    0.0137537    0.00100343      0.00112844   0.0  0.0621453
 0.000135666  0.000572523  5.28123e-5      0.000436252  0.0  0.0054991
 0.00477594   0.0874512    0.0236599       0.00429272   0.0  0.0581578
 ⋮                                      ⋱                    
 0.000713502  0.000763364  0.000580935  …  0.00413567   0.0  0.00458967
 5.02466e-6   0.0          0.0             0.000127967  0.0  0.000334776
 2.00986e-5   1.97422e-5   0.0             0.00361798

1.4. Inversa de Leontief no Modelo Fechado

In [8]:
# Inversa de Leontief do Modelo Fechado
Bf=inversa_leontief(I(size(Af)[1])-Af)
Bf

69×69 Matrix{Float64}:
 1.04926      0.100372     0.0272799    …  0.113882     0.113882
 0.014927     1.06597      0.0135345       0.0602925    0.0602925
 0.007293     0.0158079    1.05975         0.0148379    0.0148379
 0.00447054   0.00612389   0.00133688      0.00308987   0.00308987
 0.0323082    0.0358822    0.0175406       0.0506687    0.0506687
 0.000799534  0.0013772    0.000514774  …  0.00199434   0.00199434
 0.000607683  0.000681462  0.000253215     0.00111649   0.00111649
 0.0264091    0.0545602    0.0174666       0.13244      0.13244
 0.00548197   0.00815131   0.00320969      0.0166501    0.0166501
 0.0363859    0.145776     0.0462917       0.144719     0.144719
 ⋮                                      ⋱               
 0.00603855   0.007552     0.00334397   …  0.0162416    0.0162416
 0.000292115  0.00037628   0.000149966     0.000985344  0.000985344
 0.0159373    0.0230571    0.00979058      0.0773254    0.0773254
 0.000255547  0.000371346  0.000155107     0.00128203   0.001

2. Multiplicadores

2.1. Multiplicadores da Produção Simples

In [9]:
multiplicadores_produção(A,B,Bf,Setores)

68×6 Matrix{Any}:
 "Agricultura, inclusive o apoio à agricultura e a pós-colheita"             …  1.05051   0.374714  1.34672
 "Pecuária, inclusive o apoio à pecuária"                                       1.54233   0.447996  1.42781
 "Produção florestal; pesca e aquicultura"                                      0.648056  0.215949  1.16971
 "Extração de carvão mineral e de minerais não-metálicos"                       1.96104   0.467793  1.41277
 "Extração de petróleo e gás, inclusive as atividades de apoio"                 1.13795   0.384088  1.28948
 "Extração de minério de ferro, inclusive beneficiamentos e a aglomeração"   …  1.06191   0.414282  1.34019
 "Extração de minerais metálicos não-ferrosos, inclusive beneficiamentos"       1.6731    0.484923  1.41782
 "Abate e produtos de carne, inclusive os produtos do laticínio e da pesca"     1.96095   0.77389   1.70964
 "Fabricação e refino de açúcar"                                                1.98739   0.78908   1.59306
 "Outros p

2.2. Multiplicadores do Emprego Truncado

In [10]:
multiplicadores_produção_truncado(A,B,Bf,Setores)

68×6 Matrix{Any}:
 "Agricultura, inclusive o apoio à agricultura e a pós-colheita"             …  0.711177  0.374714  1.34672
 "Pecuária, inclusive o apoio à pecuária"                                       1.04414   0.447996  1.42781
 "Produção florestal; pesca e aquicultura"                                      0.438723  0.215949  1.16971
 "Extração de carvão mineral e de minerais não-metálicos"                       1.32759   0.467793  1.41277
 "Extração de petróleo e gás, inclusive as atividades de apoio"                 0.770373  0.384088  1.28948
 "Extração de minério de ferro, inclusive beneficiamentos e a aglomeração"   …  0.718894  0.414282  1.34019
 "Extração de minerais metálicos não-ferrosos, inclusive beneficiamentos"       1.13266   0.484923  1.41782
 "Abate e produtos de carne, inclusive os produtos do laticínio e da pesca"     1.32753   0.77389   1.70964
 "Fabricação e refino de açúcar"                                                1.34543   0.78908   1.59306
 "Outros p

2.3. Multiplicadores de Emprego

In [11]:
multiplicadores_emprego(e,x,B,Bf,Setores)

68×5 Matrix{Any}:
 "Agricultura, inclusive o apoio à agricultura e a pós-colheita"             …   1.21145   26.8841   1.6001
 "Pecuária, inclusive o apoio à pecuária"                                        1.18496   55.527    1.43225
 "Produção florestal; pesca e aquicultura"                                       1.15996   28.5658   1.35038
 "Extração de carvão mineral e de minerais não-metálicos"                        1.80544   22.8609   3.86781
 "Extração de petróleo e gás, inclusive as atividades de apoio"                 18.8246    11.2513  50.696
 "Extração de minério de ferro, inclusive beneficiamentos e a aglomeração"   …  13.0586    11.6134  30.2545
 "Extração de minerais metálicos não-ferrosos, inclusive beneficiamentos"        4.15315   16.9796  10.7177
 "Abate e produtos de carne, inclusive os produtos do laticínio e da pesca"      9.02458   35.8058  13.6824
 "Fabricação e refino de açúcar"                                                 5.6437    29.4792   9.71477
 "Outro

3. Índices de Ligação

Índices de Ligação para trás

In [12]:
lig_tras(B,Setores)

68×2 Matrix{Any}:
 "Agricultura, inclusive o apoio à agricultura e a pós-colheita"             …  0.953665
 "Pecuária, inclusive o apoio à pecuária"                                       1.03918
 "Produção florestal; pesca e aquicultura"                                      0.767644
 "Extração de carvão mineral e de minerais não-metálicos"                       1.04182
 "Extração de petróleo e gás, inclusive as atividades de apoio"                 0.927149
 "Extração de minério de ferro, inclusive beneficiamentos e a aglomeração"   …  0.971967
 "Extração de minerais metálicos não-ferrosos, inclusive beneficiamentos"       1.05411
 "Abate e produtos de carne, inclusive os produtos do laticínio e da pesca"     1.37586
 "Fabricação e refino de açúcar"                                                1.31969
 "Outros produtos alimentares"                                                  1.27334
 ⋮                                                                           ⋱  
 "Atividades de v

Índices de Ligação para frente

In [13]:
lig_frente(B,Setores)

68×2 Matrix{Any}:
 "Agricultura, inclusive o apoio à agricultura e a pós-colheita"             …  1.89852
 "Pecuária, inclusive o apoio à pecuária"                                       0.873513
 "Produção florestal; pesca e aquicultura"                                      0.712919
 "Extração de carvão mineral e de minerais não-metálicos"                       0.684382
 "Extração de petróleo e gás, inclusive as atividades de apoio"                 1.35186
 "Extração de minério de ferro, inclusive beneficiamentos e a aglomeração"   …  0.711683
 "Extração de minerais metálicos não-ferrosos, inclusive beneficiamentos"       0.675919
 "Abate e produtos de carne, inclusive os produtos do laticínio e da pesca"     0.758428
 "Fabricação e refino de açúcar"                                                0.668108
 "Outros produtos alimentares"                                                  1.00567
 ⋮                                                                           ⋱  
 "Atividades d

4. Extração Hipotética

In [14]:
extracao_hipotetica(A,12,y)
# Extrai o  décimo-segundo setor desta economia, qual seja o setor: "Fabricação de produtos do fumo"
# Retorna dois valores: primeiro, valor absoluto e, depois, o percentual de redução em relação à situação inicial

1×2 Matrix{Float64}:
 17544.4  0.146083